In [ ]:
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import os, random
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import (Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization, 
                                    GlobalAveragePooling2D, UpSampling2D, Input, AlphaDropout)
from tensorflow.keras.mixed_precision import experimental as mixed_precision
from tensorflow.keras.applications.efficientnet import EfficientNetB7
from tensorflow.keras.callbacks import ReduceLROnPlateau, LearningRateScheduler, EarlyStopping, ModelCheckpoint
#from tensorflow.keras.optimizers.schedules import PolynomialDecay
from kaggle_datasets import KaggleDatasets
from tensorflow.keras.regularizers import L2
from tensorflow.keras import Sequential
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from IPython.display import clear_output
from tensorflow.keras.layers.experimental import preprocessing
from sklearn.model_selection import train_test_split
import PIL, PIL.ImageOps, PIL.ImageEnhance, PIL.ImageDraw
from PIL import Image

#gcs = KaggleDatasets().get_gcs_path("tfrecordgdsc")

In [ ]:
test = pd.read_csv("../input/gdsc-ai-challenge/sampleSubmission.csv")
train = pd.read_csv("../input/gdsc-ai-challenge/trainLabels.csv")

train["id"] = train["id"].astype(str)
test["id"] = test["id"].astype(str)

test = test.drop(columns = "label")

In [ ]:
train_path = "../input/gdsc-ai-challenge/train/train/"
test_path = "../input/gdsc-ai-challenge/test/test/"

In [ ]:
train["id"] = [train_path] + train["id"].values + [".png"]
test["id"] = [test_path] + test["id"].values + [".png"]

In [ ]:
labels = sorted(set(train["label"].values))
encoded_label = pd.get_dummies(labels).values
map_label = dict(zip(labels, encoded_label))
train["encoded_label"] = train["label"].map(map_label)

# Reading the data
training_images = np.array([plt.imread(f"{train_path}{i + 1}.png") for i in range(0, 50000)])
testing_images  = np.array([plt.imread(f"{test_path}{i + 1}.png") for i in range(0, 20000)])
    
training_images = training_images.astype("float32")
testing_images = testing_images.astype("float32")   
    
#training_images /= 255.0
#testing_images /= 255.0

In [ ]:
#training_images = tf.keras.applications.inception_resnet_v2.preprocess_input(training_images)
#testing_images = tf.keras.applications.inception_resnet_v2.preprocess_input(testing_images)

In [ ]:
mappings, labeling_images = np.unique(train["label"].to_numpy(copy = True), return_inverse = True)
training_labels = labeling_images.reshape(-1, 1)

labeling_images = tf.keras.utils.to_categorical(training_labels, 10)
labeling_images

In [ ]:
def ShearX(img, v):
    assert -0.3 <= v <= 0.3
    if random.random() > 0.5:
        v = -v
    return img.transform(img.size, PIL.Image.AFFINE, (1, v, 0, 0, 1, 0))

def ShearY(img, v):
    assert -0.3 <= v <= 0.3
    if random.random() > 0.5:
        v = -v
    return img.transform(img.size, PIL.Image.AFFINE, (1, 0, 0, v, 1, 0))

def TranslateX(img, v):
    assert -0.45 <= v <= 0.45
    if random.random() > 0.5:
        v = -v
    v = v * img.size[0]
    return img.transform(img.size, PIL.Image.AFFINE, (1, 0, v, 0, 1, 0))

def TranslateXabs(img, v):
    assert 0 <= v
    if random.random() > 0.5:
        v = -v
    return img.transform(img.size, PIL.Image.AFFINE, (1, 0, v, 0, 1, 0))

def TranslateY(img, v):
    assert -0.45 <= v <= 0.45
    if random.random() > 0.5:
        v = -v
    v = v * img.size[1]
    return img.transform(img.size, PIL.Image.AFFINE, (1, 0, 0, 0, 1, v))

def TranslateYabs(img, v):
    assert 0 <= v
    if random.random() > 0.5:
        v = -v
    return img.transform(img.size, PIL.Image.AFFINE, (1, 0, 0, 0, 1, v))

def Rotate(img, v):
    assert -30 <= v <= 30
    if random.random() > 0.5:
        v = -v
    return img.rotate(v)

def AutoContrast(img, _):
    return PIL.ImageOps.autocontrast(img)

def Invert(img, _):
    return PIL.ImageOps.invert(img)

def Equalize(img, _):
    return PIL.ImageOps.equalize(img)

def Mirror(img, _):
    return PIL.ImageOps.mirror(img)

def Solarize(img, v):
    assert 0 <= v <= 256
    return PIL.ImageOps.solarize(img, v)

def SolarizeAdd(img, addition=0, threshold=128):
    img_np = np.array(img).astype(np.int)
    img_np = img_np + addition
    img_np = np.clip(img_np, 0, 255)
    img_np = img_np.astype(np.uint8)
    img = Image.fromarray(img_np)
    return PIL.ImageOps.solarize(img, threshold)

def Posterize(img, v):
    v = int(v)
    v = max(1, v)
    return PIL.ImageOps.posterize(img, v)

def Contrast(img, v):
    assert 0.1 <= v <= 1.9
    return PIL.ImageEnhance.Contrast(img).enhance(v)

def Color(img, v):
    assert 0.1 <= v <= 1.9
    return PIL.ImageEnhance.Color(img).enhance(v)

def Brightness(img, v):
    assert 0.1 <= v <= 1.9
    return PIL.ImageEnhance.Brightness(img).enhance(v)

def Sharpness(img, v):
    assert 0.1 <= v <= 1.9
    return PIL.ImageEnhance.Sharpness(img).enhance(v)

def Cutout(img, v):
    assert 0.0 <= v <= 0.2
    if v <= 0.:
        return img

    v = v * img.size[0]
    return CutoutAbs(img, v)

def CutoutAbs(img, v):
    if v < 0:
        return img
    w, h = img.size
    x0 = np.random.uniform(w)
    y0 = np.random.uniform(h)

    x0 = int(max(0, x0 - v / 2.))
    y0 = int(max(0, y0 - v / 2.))
    x1 = min(w, x0 + v)
    y1 = min(h, y0 + v)

    xy = (x0, y0, x1, y1)
    color = (125, 123, 114)
    # color = (0, 0, 0)
    img = img.copy()
    PIL.ImageDraw.Draw(img).rectangle(xy, color)
    return img

def SamplePairing(imgs):
    def f(img1, v):
        i = np.random.choice(len(imgs))
        img2 = PIL.Image.fromarray(imgs[i])
        return PIL.Image.blend(img1, img2, v)
    return f

def Identity(img, v):
    return img

def augment_list():
    l = [
        (AutoContrast, 0, 1),
        (Equalize, 0, 1),
        (Invert, 0, 1),
        (Rotate, 0, 30),
        (Posterize, 0, 4),
        (Solarize, 0, 256),
        (SolarizeAdd, 0, 110),
        (Color, 0.1, 1.9),
        (Contrast, 0.1, 1.9),
        (Brightness, 0.1, 1.9),
        (Sharpness, 0.1, 1.9),
        (ShearX, 0., 0.3),
        (ShearY, 0., 0.3),
        (CutoutAbs, 0, 40),
        (TranslateXabs, 0., 100),
        (TranslateYabs, 0., 100),
        (Mirror, 0., 0.)
    ]
    return l

class RandAugment:
    def __init__(self, n, m):
        self.n = n
        self.m = m
        self.augment_list = augment_list()

    def __call__(self, img): # img shape: [batch_size, 32, 32, 3]
        for i in range(img.shape[0]):
            img_temp = Image.fromarray(np.uint8(img[i])).convert("RGB")
            ops = random.choices(self.augment_list, k = self.n)
            for op, minval, maxval in ops:
                val = (float(self.m) / 30) * float(maxval - minval) + minval
                img[i] = np.array(op(img_temp, val), dtype = "float32")

        return img

In [ ]:
def CNN():
    cnn = Sequential()
    
    cnn.add(Conv2D(filters = 32, kernel_size = 3, padding = "same", activation = "relu", input_shape = (32, 32, 3)))
    cnn.add(BatchNormalization())
    cnn.add(Conv2D(filters = 32, kernel_size = 3, padding = "same", activation = "relu"))
    cnn.add(BatchNormalization())
    cnn.add(MaxPooling2D(pool_size = 2))
    cnn.add(Dropout(0.3))

    cnn.add(Conv2D(filters = 64, kernel_size = 3, padding = "same", activation = "relu"))
    cnn.add(BatchNormalization())
    cnn.add(Conv2D(filters = 64, kernel_size = 3, padding = "same", activation = "relu"))
    cnn.add(BatchNormalization())
    cnn.add(MaxPooling2D(pool_size = 2))
    cnn.add(Dropout(0.4))

    cnn.add(Conv2D(filters = 128, kernel_size = 3, padding = "same", activation = "relu"))
    cnn.add(BatchNormalization())
    cnn.add(Conv2D(filters = 128, kernel_size = 3, padding = "same", activation = "relu"))
    cnn.add(BatchNormalization())
    cnn.add(MaxPooling2D(pool_size = 2))
    cnn.add(Dropout(0.4))

    cnn.add(Flatten())
    cnn.add(Dense(128, activation = "relu"))
    cnn.add(BatchNormalization())
    cnn.add(Dropout(0.4))
    cnn.add(Dense(len(labels), activation = "softmax"))
    return cnn

def CNN2():
    weight_decay = 0.0005
    cnn  =  Sequential()

    cnn.add(Conv2D(64, (3, 3), padding = "same", input_shape  =  (32, 32, 3), 
                   kernel_regularizer = L2(weight_decay), activation = "relu"))
    cnn.add(BatchNormalization())
    cnn.add(Dropout(0.3))

    cnn.add(Conv2D(64, (3, 3), padding = "same", kernel_regularizer = L2(weight_decay), activation = "relu"))
    cnn.add(BatchNormalization())
    cnn.add(MaxPooling2D(pool_size = (2, 2)))

    cnn.add(Conv2D(128, (3, 3), padding = "same", kernel_regularizer = L2(weight_decay), activation = "relu"))
    cnn.add(BatchNormalization())
    cnn.add(Dropout(0.4))

    cnn.add(Conv2D(128, (3, 3), padding = "same", kernel_regularizer = L2(weight_decay), activation = "relu"))
    cnn.add(BatchNormalization())
    cnn.add(MaxPooling2D(pool_size = (2, 2)))

    cnn.add(Conv2D(256, (3, 3), padding = "same", kernel_regularizer = L2(weight_decay), activation = "relu"))
    cnn.add(BatchNormalization())
    cnn.add(Dropout(0.4))

    cnn.add(Conv2D(256, (3, 3), padding = "same", kernel_regularizer = L2(weight_decay), activation = "relu"))
    cnn.add(BatchNormalization())
    cnn.add(Dropout(0.4))

    cnn.add(Conv2D(256, (3, 3), padding = "same", kernel_regularizer = L2(weight_decay), activation = "relu"))
    cnn.add(BatchNormalization())
    cnn.add(MaxPooling2D(pool_size = (2, 2)))


    cnn.add(Conv2D(512, (3, 3), padding = "same", kernel_regularizer = L2(weight_decay), activation = "relu"))
    cnn.add(BatchNormalization())
    cnn.add(Dropout(0.4))

    cnn.add(Conv2D(512, (3, 3), padding = "same", kernel_regularizer = L2(weight_decay), activation = "relu"))
    cnn.add(BatchNormalization())
    cnn.add(Dropout(0.4))

    cnn.add(Conv2D(512, (3, 3), padding = "same", kernel_regularizer = L2(weight_decay), activation = "relu"))
    cnn.add(BatchNormalization())

    cnn.add(MaxPooling2D(pool_size = (2, 2)))


    cnn.add(Conv2D(512, (3, 3), padding = "same", kernel_regularizer = L2(weight_decay), activation = "relu"))
    cnn.add(BatchNormalization())
    cnn.add(Dropout(0.4))

    cnn.add(Conv2D(512, (3, 3), padding = "same", kernel_regularizer = L2(weight_decay), activation = "relu"))
    cnn.add(BatchNormalization())
    cnn.add(Dropout(0.4))

    cnn.add(Conv2D(512, (3, 3), padding = "same", kernel_regularizer = L2(weight_decay), activation = "relu"))
    cnn.add(BatchNormalization())
    cnn.add(MaxPooling2D(pool_size = (2, 2)))
    cnn.add(Dropout(0.5))

    cnn.add(Flatten())
    cnn.add(Dense(512, kernel_regularizer = L2(weight_decay), activation = "relu"))
    cnn.add(BatchNormalization())
    cnn.add(Dropout(0.5))
    cnn.add(Dense(len(labels), activation = "softmax"))
    return cnn

def CNN3():
    inputs = Input(shape = (32, 32, 3))
    
    resize = UpSampling2D(size = (7, 7))(inputs)
    resnet = tf.keras.applications.resnet.ResNet50(input_shape = (224, 224, 3), include_top = False,
                        weights = "imagenet")(resize)
    resnet.trainable = False
    
    x = GlobalAveragePooling2D()(resnet)
    #x = Flatten()(x)
    x = Dense(1024, activation = "relu")(x)
    x = Dense(512, activation = "relu")(x)
    x = Dense(len(labels), activation = "softmax")(x)
    
    cnn = tf.keras.Model(inputs = inputs, outputs = x)
    return cnn
    
def CNN4():
    inputs = Input(shape = (32, 32, 3))
    
    resize = UpSampling2D(size = (7, 7))(inputs)
    resnet = tf.keras.applications.vgg16.VGG16(input_shape = (224, 224, 3), include_top = False, 
                            weights = "imagenet")(resize)
    resnet.trainable = False
    
    x = GlobalAveragePooling2D()(resnet)
    #x = Flatten()(x)
    x = Dense(1024, activation = "relu")(x)
    x = Dense(512, activation = "relu")(x)
    x = Dense(len(labels), activation = "softmax")(x)
    
    cnn = tf.keras.Model(inputs = inputs, outputs = x)
    return cnn

def CNN5():
    inputs = Input(shape = (32, 32, 3))
    
    resize = UpSampling2D(size = (7, 7))(inputs)
    resnet = tf.keras.applications.resnet.ResNet152(input_shape = (224, 224, 3), include_top = False,
                        weights = "imagenet")(resize)
    resnet.trainable = False
    
    x = GlobalAveragePooling2D()(resnet)
    #x = Flatten()(x)
    #x = Dense(1024, activation = "relu")(x)
    x = Dense(512, activation = "relu")(x)
    x = Dense(len(labels), activation = "softmax")(x)
    
    cnn = tf.keras.Model(inputs = inputs, outputs = x)
    return cnn

def CNN6():
    with strategy.scope():
        inputs = Input(shape = (32, 32, 3))

        resize = UpSampling2D(size = (7, 7))(inputs)
        eff = tf.keras.applications.efficientnet.EfficientNetB6(input_shape = (224, 224, 3), include_top = False,
                            weights = "imagenet")(resize)
        eff.trainable = False

        x = GlobalAveragePooling2D()(eff)
        #x = Flatten()(x)
        #x = Dense(1024, activation = "relu")(x)
        x = Dense(512, activation = "relu")(x)
        x = Dense(len(labels), activation = "softmax")(x)

        cnn = tf.keras.Model(inputs = inputs, outputs = x)
    return cnn

def CNN7():
    with strategy.scope():
        inputs = Input(shape = (32, 32, 3))

        #augment = Augmentation(inputs)
        
        resize = UpSampling2D(size = (7, 7))(inputs)
        resnet = tf.keras.applications.inception_resnet_v2.InceptionResNetV2(input_shape = (224, 224, 3), 
                                                                             include_top = False,
        weights = "imagenet")(resize)
        resnet.trainable = False

        x = GlobalAveragePooling2D()(resnet)
        #x = Flatten()(x)
        #x = tf.keras.layers.GaussianNoise(stddev = 0.1)(x)
        x = Dense(1024, activation = "relu")(x)
        x = Dropout(0.2)(x)
        x = Dense(512, activation = "relu")(x)
        x = Dense(len(labels), activation = "softmax")(x)

        cnn = tf.keras.Model(inputs = inputs, outputs = x)
    return cnn

def CNN8():
    # https://tfhub.dev/google/bit/m-r152x4/1
    # https://tfhub.dev/google/bit/m-r50x1/1
    # https://tfhub.dev/google/bit/m-r101x3/1
    module = tf.keras.models.load_model("../input/bit-model/bit_m-r50x1_1")
    with strategy.scope():
        inputs = Input(shape = (32, 32, 3))

        augment = Augmentation(inputs)
        
        resize = UpSampling2D(size = (5, 5))(augment)
        
        features = module(resize)
        
        x = Dense(1024, activation = "relu")(features)
        x = Dropout(0.2)(x)
        x = Dense(512, activation = "relu")(x)
        x = Dense(len(labels), activation = "softmax")(x)

        cnn = tf.keras.Model(inputs = inputs, outputs = x)
    return cnn

def Augmentation(x):
    #aug = RandAugment(n = 2, m = 14)
    x = tf.image.flip_left_right(x)
    #x = tf.image.flip_up_down(x)
    #x = tf.image.random_flip_left_right(x)
    #x = tf.image.random_flip_up_down(x)
    #x = tf.keras.layers.GaussianNoise(stddev = 0.05)(x, training = True)
    #x = preprocessing.RandomFlip(mode = "horizontal_and_vertical")(x, training = True)
    #x = preprocessing.RandomRotation(factor = (-0.25, 0.25), fill_mode = "reflect")(x, training = True)
    x = preprocessing.RandomTranslation(height_factor = (-0.2, 0.2), width_factor = (-0.3, 0.3), 
                                        fill_mode = "reflect")(x, training = True)
    #x = preprocessing.RandomZoom(height_factor = (-0.1, 0.1), width_factor = (-0.1, 0.1))(x, training = True)
    #x = tf.image.random_saturation(x, -0.2, 0.2)
    #x = aug(x)
    return x

def sample_beta_distribution(size, concentration_0 = 0.2, concentration_1 = 0.2):
    gamma_1_sample = tf.random.gamma(shape = [size], alpha = concentration_1)
    gamma_2_sample = tf.random.gamma(shape = [size], alpha = concentration_0)
    return gamma_1_sample / (gamma_1_sample + gamma_2_sample)

def MixUp(images_one, images_two, labels_one, labels_two, alpha = 0.2):
    l = sample_beta_distribution(size = 50000, concentration_0 = alpha, concentration_1 = alpha)
    x_l = tf.reshape(l, (50000, 1, 1, 1))
    y_l = tf.reshape(l, (50000, 1))

    images = images_one * x_l + images_two * (1 - x_l)
    labels = labels_one * y_l + labels_two * (1 - y_l)
    return (np.array(images), np.array(labels))

In [ ]:
class PolynomialDecay():
    def __init__(self, maxEpochs = 100, initLR = 0.01, endLR = 0.001, power = 1.0):
        self.maxEpochs = maxEpochs
        self.initLR = initLR
        self.endLR = endLR
        self.power = power
    def __call__(self, epoch):
        decay = (1 - (epoch / float(self.maxEpochs))) ** self.power
        alpha = (self.initLR - self.endLR) * decay + self.endLR
        return float(alpha)
    
class StepDecay():
    def __init__(self, initial_LR = 0.01, factor = 0.25, drop_every = 10):
        self.initial_LR = initial_LR
        self.factor = factor
        self.drop_every = drop_every

    def __call__(self, epoch):
        exp = np.floor((1 + epoch) / self.drop_every)
        new_alpha = self.initial_LR * (self.factor ** exp)

        return float(new_alpha)

def CustomDecay(epoch, lr):
      if epoch < 200:
        return lr
      else: # epoch 200th
        factor = (0.001 / 0.00001) / (500 - 200)
        return lr * factor

In [ ]:
image_size = (32, 32)
def decode_image(image_data):
    image = tf.image.decode_png(image_data, channels = 3)
    image = tf.cast(image, tf.float32) / 255.0  # convert image to floats in [0, 1] range
    image = tf.image.resize(image, image_size)
    return image

def read_labeled_tfrecord(example):
    data = (
        {
          'image' : tf.io.FixedLenFeature([], tf.string),
          'label': tf.io.FixedLenFeature([], tf.int64)
        }
    )
    example = tf.io.parse_single_example(example, data)
    image = decode_image(example['image'])
    label = tf.one_hot(example['label'], 10)
    label = tf.cast(label, tf.float32)
    return image, label

def read_unlabeled_tfrecord(example):
    data = (
        {
          'image' : tf.io.FixedLenFeature([], tf.string)
        }
    )
    example = tf.io.parse_single_example(example, data)
    image = decode_image(example['image'])
    return image

def load_dataset(filenames, labeled = True, ordered = False):
    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False # disable order, increase speed

    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads = tf.data.experimental.AUTOTUNE)
    dataset = dataset.with_options(ignore_order)
    dataset = dataset.map(read_labeled_tfrecord if labeled else read_unlabeled_tfrecord)
    if labeled:
        dataset = dataset.map(Preprocess)

    return dataset

def get_training_dataset(filenames, batch_size = 64):
    dataset = load_dataset(filenames, labeled = True)
    dataset = dataset.repeat()
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
    return dataset

def get_test_dataset(filenames, batch_size = 64):
    dataset = load_dataset(filenames, labeled = False, ordered = True)
    dataset = dataset.repeat()
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
    return dataset

def Preprocess(image, label):
    image = tf.image.random_flip_left_right(image)
    return image, label

#training_filenames = tf.io.gfile.glob(f'{gcs}/train*.tfrec')
#validation_filenames = tf.io.gfile.glob(f'{gcs}/validation*.tfrec')
#test_filenames = tf.io.gfile.glob(f'{gcs}/test*.tfrec')

#training_set = get_training_dataset(training_filenames, batch_size = 40)
#validation_set = get_training_dataset(validation_filenames, batch_size = 40)

In [ ]:
#validation_split = 0.1
#batch_size = 64
#preprocess_vgg = tf.keras.applications.vgg16.preprocess_input
#preprocess_resnet = tf.keras.applications.resnet50.preprocess_input
#preprocess_effb7 = tf.keras.applications.efficientnet.preprocess_input
#preprocess_incep = tf.keras.applications.inception_resnet_v2.preprocess_input

#traingen = ImageDataGenerator(rotation_range = 10, width_shift_range = 0.05, height_shift_range = 0.05,
#                              horizontal_flip = True, rescale = 1./255, zoom_range = 0.1, shear_range = 10, 
#                              validation_split = validation_split)
#traingen = ImageDataGenerator(rescale = 1./255, validation_split = validation_split, 
#                              preprocessing_function = preprocess_incep)
#traingen.fit(training_images)
#training_set_gen = traingen.flow_from_dataframe(dataframe = train, x_col = "id", validate_filenames = False, 
#                                                y_col = "label", target_size = (32, 32), batch_size = batch_size,
#                                                subset = "training", class_mode = "categorical") 
#validation_set_gen = traingen.flow_from_dataframe(dataframe = train, x_col = "id", validate_filenames = False, 
#                                                  y_col = "label", target_size = (32, 32), batch_size = batch_size,
#                                                  subset = "validation", class_mode = "categorical") 

#testgen = ImageDataGenerator(rescale = 1./255, preprocessing_function = preprocess_incep)
#test_set_gen = testgen.flow_from_dataframe(dataframe = test, x_col = "id", validate_filenames = False,
#                                            class_mode = None, target_size = (32, 32), batch_size = batch_size, 
#                                            shuffle = False)

In [ ]:
training_images *= 255
testing_images *= 255
#mixup, mixup_label = MixUp(training_images, training_images[::-1], labeling_images, labeling_images[::-1], alpha = 0.2)

In [ ]:
nets = 4
labels_pred = np.zeros((test.shape[0], 10))

labeling_images_temp = np.concatenate((labeling_images, labeling_images), axis = 0)
    
for i in range(nets):
    training_images_temp = np.concatenate((training_images, np.array(Augmentation(training_images))), axis = 0)
    try:
        TPU = tf.distribute.cluster_resolver.TPUClusterResolver()
        print('Running on TPU ', TPU.master())
    except ValueError:
        print('Running on GPU or CPU')
        TPU = None

    if TPU:
        tf.config.experimental_connect_to_cluster(TPU)
        tf.tpu.experimental.initialize_tpu_system(TPU)
        strategy = tf.distribute.experimental.TPUStrategy(TPU)
    else:
        strategy = tf.distribute.get_strategy()
        policy = mixed_precision.Policy('mixed_float16')
        mixed_precision.set_policy(policy)
        tf.config.optimizer.set_jit(True)
    #==============================================
    po_schedule = PolynomialDecay(maxEpochs = 150, initLR = 0.01, endLR = 0.001, power = 5.0)
    step_schedule = StepDecay(initial_LR = 0.01, factor = 0.25, drop_every = 10)

    opt_sgd = tf.keras.optimizers.SGD(learning_rate = 0.1 if TPU else 0.001, 
                                      momentum = 0.9) # momentum = 0.9, learning_rate = 0.1
    opt_adam = tf.keras.optimizers.Adam(learning_rate = 0.01, beta_1 = 0.9, beta_2 = 0.999, epsilon = 1e-07)
    opt_rmspop = tf.keras.optimizers.RMSprop(learning_rate = 0.01, rho = 0.9, momentum = 0.0, epsilon = 1e-07)
    #==============================================
    reduce_lr = ReduceLROnPlateau(factor = 0.9, patience = 5, min_lr = 1e-10, verbose = 1)
    early_stopping = EarlyStopping(monitor='val_loss', patience = 10, verbose = 1, mode = "min", 
                                   restore_best_weights = True)
    lr_schedule = LearningRateScheduler(CustomDecay, verbose = 1)
    checkpoint = ModelCheckpoint(filepath = f"./CNN{i + 1}.h5", save_weights_only = True, 
                                 monitor = "val_accuracy", mode = "max", save_best_only = True)
    #==============================================
    clear_output(wait = True)
    print(f"Current net: {i + 1}/{nets}")

    cnn = CNN6() # effb6
    cnn.compile(optimizer = opt_sgd, loss = tf.keras.losses.CategoricalCrossentropy(from_logits = False), 
                   metrics = ["accuracy"], steps_per_execution = 2048 if TPU else 1)
    #cnn.summary()

    #cnn.fit(training_set, validation_data = validation_set, epochs = 100, steps_per_epoch = 1125, validation_steps = 125)
    cnn.fit(training_images_temp, labeling_images_temp, validation_split = 0.1, epochs = 100 if TPU else 100, 
            callbacks = [reduce_lr], batch_size = 256 if TPU else 64)
    #cnn.fit(traingen.flow(x_train, y_train, batch_size = batch_size, subset = "training"), 
    #        validation_data = traingen.flow(x_train, y_train, batch_size = 64, subset = "validation"), 
    #        epochs = 50, callbacks = [reduce_lr])
    #cnn.fit(training_set_gen, validation_data = validation_set_gen, epochs = 50, callbacks = [reduce_lr])
    #cnn.fit(x_train, y_train, validation_split = 0.1, epochs = 100 if TPU else 100, 
    #        batch_size = 1024 if TPU else 64, callbacks = [reduce_lr])

    #cnn.load_weights(f"./CNN{i + 1}.h5")
    labels_pred += cnn.predict(testing_images, verbose = 1)
    #labels_pred += cnn.predict(test_set_gen, verbose = 1)
    
    cnn.save_weights(f"./CNN{i + 1}.h5")
    #================================================
    clear_output(wait = True)

labels_pred = np.argmax(labels_pred, axis = 1)
labels_pred

In [ ]:
labels_name = []
for i in range(len(labels_pred)):
    labels_name.append(labels[labels_pred[i]])

In [ ]:
submission = pd.read_csv("../input/gdsc-ai-challenge/sampleSubmission.csv").drop(columns = "label")
submission["label"] = labels_name
submission.to_csv("./submission.csv", index = False)

In [ ]:
submission

In [ ]:
#cnn.save("./CNN.h5")
#cnn = tf.keras.models.load_model("./CNN.h5")